In [1]:
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
kiva_df = pd.read_csv("data/kiva_cleaned.csv")
kiva_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6818 entries, 0 to 6817
Data columns (total 9 columns):
loan_id        6818 non-null int64
status         6818 non-null object
sector         6818 non-null object
en             6818 non-null object
country        6818 non-null object
gender         6818 non-null object
loan_amount    6818 non-null float64
nonpayment     6818 non-null object
en_clean       6802 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 479.5+ KB


In [3]:
kiva_df = kiva_df.dropna()
kiva_df = kiva_df.reset_index(drop=True)

In [4]:
kiva_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6802 entries, 0 to 6801
Data columns (total 9 columns):
loan_id        6802 non-null int64
status         6802 non-null object
sector         6802 non-null object
en             6802 non-null object
country        6802 non-null object
gender         6802 non-null object
loan_amount    6802 non-null float64
nonpayment     6802 non-null object
en_clean       6802 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 478.3+ KB


In [5]:
kiva_df.head()

,loan_id,status,sector,en,country,gender,loan_amount,nonpayment,en_clean
0,0,defaulted,Agriculture,<i>This description is written by Rowland Amul...,Kenya,M,500.0,lender,"Robert, 40, is married and has 6 children. In ..."
1,1,defaulted,Food,<i>This description is written by Rowland Amul...,Kenya,F,500.0,lender,"Petronilla, 30, was deserted by her husband an..."
2,2,defaulted,Food,<i>This description was written by Richard Maz...,Kenya,M,500.0,lender,"Tom Mung'ahu, 45, is married and has 6 childre..."
3,3,defaulted,Services,<i>This description was written by Rowland Amu...,Kenya,F,500.0,lender,"Benedina, 42, is married and has 4 girls. In a..."
4,4,defaulted,Construction,<i>This description was written by Rowland Amu...,Kenya,M,500.0,lender,"Vincent Ondego 40, is married and has 8 childr..."


In [6]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import unidecode

stop_words = set(stopwords.words('english') + stopwords.words('spanish'))

lemmer = WordNetLemmatizer()

def preprocess(x):
    x = x.lower()
    
    x = re.sub(r'[^\w\s]', '', x)
    
    x = unidecode.unidecode(x)
    
    x = re.sub(r'\d+', '', x)
    
    x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]

    return ' '.join(x)

kiva_df['en_clean_pre'] = kiva_df['en_clean'].apply(preprocess)

In [7]:
kiva_df.head()

,loan_id,status,sector,en,country,gender,loan_amount,nonpayment,en_clean,en_clean_pre
0,0,defaulted,Agriculture,<i>This description is written by Rowland Amul...,Kenya,M,500.0,lender,"Robert, 40, is married and has 6 children. In ...",robert married child addition family take care...
1,1,defaulted,Food,<i>This description is written by Rowland Amul...,Kenya,F,500.0,lender,"Petronilla, 30, was deserted by her husband an...",petronilla deserted husband responsible upbrin...
2,2,defaulted,Food,<i>This description was written by Richard Maz...,Kenya,M,500.0,lender,"Tom Mung'ahu, 45, is married and has 6 childre...",tom mungahu married child child attending elem...
3,3,defaulted,Services,<i>This description was written by Rowland Amu...,Kenya,F,500.0,lender,"Benedina, 42, is married and has 4 girls. In a...",benedina married girl addition family also tak...
4,4,defaulted,Construction,<i>This description was written by Rowland Amu...,Kenya,M,500.0,lender,"Vincent Ondego 40, is married and has 8 childr...",vincent ondego married child beside family vin...


In [8]:
kiva_df.iloc[0, :].en_clean

'Robert, 40, is married and has 6 children. In addition to his family of 8, he takes care of his mother and 5 brothers. Robert started by planting vegetables and selling at the local market. He then diversified with a tea nursery which is more profitable. Given a loan of $500, Robert will be able to improve his activities by buying fertilizers, pesticides, a pump and seedlings. The pump will remove the uncertainties of the weather. The current capital in his venture is Ksh 10500 or $150, which does not allow Robert to maximize his potential. Robert completed high school and has never got employment. He got apprentice training from a tea extension officer. He is a go getter, and his main hobby is teaching music.'

In [9]:
kiva_df.iloc[0, :].en_clean_pre

'robert married child addition family take care mother brother robert started planting vegetable selling local market diversified tea nursery profitable given loan robert able improve activity buying fertilizer pesticide pump seedling pump remove uncertainty weather current capital venture ksh allow robert maximize potential robert completed high school never got employment got apprentice training tea extension officer go getter main hobby teaching music'

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df = 0.5, min_df=0.05, max_features = 500, ngram_range=[1,3])
dtm = vectorizer.fit_transform(kiva_df['en_clean_pre'])

In [11]:
dtm.shape

(6802, 270)

In [12]:
vectorizer.get_feature_names()

['able',
 'active',
 'active member',
 'activity',
 'add',
 'additional',
 'age',
 'ago',
 'allow',
 'also',
 'always',
 'amount',
 'another',
 'applied',
 'applied loan',
 'area',
 'attend',
 'back',
 'basic',
 'bean',
 'began',
 'belief',
 'better',
 'born',
 'bought',
 'boy',
 'business selling',
 'buy',
 'buying',
 'capital',
 'care',
 'cement',
 'child school',
 'childrens',
 'church',
 'city',
 'client',
 'clothes',
 'clothing',
 'college',
 'community',
 'continue',
 'cost',
 'could',
 'cow',
 'currently',
 'customer',
 'dairy',
 'dairy cow',
 'daughter',
 'day',
 'decided',
 'demand',
 'different',
 'domingo',
 'dominican',
 'dream',
 'due',
 'earn',
 'educate',
 'education',
 'enable',
 'enough',
 'entrepreneur',
 'every',
 'every day',
 'expand',
 'expand business',
 'expense',
 'experience',
 'explains',
 'family',
 'farm',
 'farmer',
 'farming',
 'father',
 'fee',
 'feed',
 'first',
 'first loan',
 'five',
 'five child',
 'food',
 'four',
 'four child',
 'friend',
 'fruit',

In [13]:
bow_df = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names(), index=kiva_df.index)

kiva_df_bow = pd.concat([kiva_df, bow_df], axis=1)
kiva_df_bow.shape

(6802, 280)